# People Analytics
## Predicting Employee Attrition with R

<img src="images/office-and-workers-in-barcelona-spain.jpg" />

<br>

*In this notebook, we will walk through solving a classification problem using machine learning. To do so, we will introduce the Scikit-Learn machine learning library for Python.*

### Table of Contents

<a href="#sectioncase">Case Study: Employee Attrition at IBM</a>

<a href="#sectiondata"> The Dataset </a>

<a href="#sectionlogistic"> Logistic Regression </a>

<ol start="9">
    <li><a href="#section9">Machine Learning</a>
        <ol type=a>
            <br>
            <li><a href="#section9a">The K-Nearest Neighbors Algorithm</a></li>
            <br>
            <li><a href="#section9b">Using Scikit-Learn: An Example</a></li>
            <br>
            <li><a href="#section9c">Using Scikit-Learn: KNN</a></li>            
        </ol>
    </li>
    </ol>

## <a id= "sectioncase">Case Study: Employee Attrition at IBM </a>

*Employee attrition* refers to a decrease in employees at a company where the company does not intend to fill the vacancy afterward. Attrition, caused by resignations, retirements, or the elimination of job positions, is important for businesses to anticipate so they can keep costs low and properly distribute workloads.

Today, we'll try to predict whether or not an employee will leave their job using [a data set created by IBM](https://www.kaggle.com/pavansubhasht/ibm-hr-analytics-attrition-dataset/home). Because employee data is confidential, IBM's data set is fictional- it does not represent real employees. However, the data and the problem are structured very similarly to how actual companies such as IBM are approaching this very real problem.

<div class="alert alert-info">
    <b>NOTE</b>: we've made some changes to the data set to make this assignment simpler- things like renaming values, dropping confusing features, and adding definitions for columns. Please refer to the data dictionary below and <b>not</b> the one at the link when completing this assignment.
    </div>

Run the following cell to load the data.

## <a name="sectiondata"> The Dataset </a>


In [46]:
# load the dataset
hr = read.csv("data/HR-Employee-Attrition.csv")

# show the first 6 rows of the dataset
head(hr)

Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,⋯,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
<int>,<fct>,<fct>,<int>,<fct>,<int>,<int>,<fct>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,⋯,1,80,0,8,0,1,6,4,0,5
49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,⋯,4,80,1,10,3,3,10,7,1,7
37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,⋯,2,80,0,7,3,3,0,0,0,0
33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,⋯,3,80,0,8,3,3,8,7,3,0
27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,⋯,4,80,1,6,3,3,2,2,2,2
32,No,Travel_Frequently,1005,Research & Development,2,2,Life Sciences,1,8,⋯,3,80,0,8,2,2,7,7,3,6


In [7]:
names(hr)

[1] "Age"                      "Attrition"               
 [3] "BusinessTravel"           "DailyRate"               
 [5] "Department"               "DistanceFromHome"        
 [7] "Education"                "EducationField"          
 [9] "EmployeeCount"            "EmployeeNumber"          
[11] "EnvironmentSatisfaction"  "Gender"                  
[13] "HourlyRate"               "JobInvolvement"          
[15] "JobLevel"                 "JobRole"                 
[17] "JobSatisfaction"          "MaritalStatus"           
[19] "MonthlyIncome"            "MonthlyRate"             
[21] "NumCompaniesWorked"       "Over18"                  
[23] "OverTime"                 "PercentSalaryHike"       
[25] "PerformanceRating"        "RelationshipSatisfaction"
[27] "StandardHours"            "StockOptionLevel"        
[29] "TotalWorkingYears"        "TrainingTimesLastYear"   
[31] "WorkLifeBalance"          "YearsAtCompany"          
[33] "YearsInCurrentRole"       "YearsSinceLastPromotion" 
[35] "YearsWithCurrManager"

This data set contains a wealth of information that could be used to predict attrition. Here's the **data dictionary**: the information about each feature (column) contained in the data set.


* `Age`: employee's age
* `Attrition`: the **response variable**. 1 if employee left IBM due to normal life circumstances, 0 if the employee stayed with IBM
* `BusinessTravel`: the amount of travel the employee does for business. 0 (no travel), 1 (rare travel), or 2 (frequent travel)
* `Department`: employee's home department. 'Sales', 'Research & Development', or 'Human Resources'
* `DistanceFromHome`: number of miles the employee has to travel to work
* `Education`: employee's highest level of education. 1 (below college), 2 (some college), 3 (bachelor's degree), 4 (master's degree), 5 (doctorate)
* `Gender`: 1 (female) or 0 (male)
* `JobRole`: 'Sales Executive', 'Research Scientist', 'Laboratory Technician', 'Manufacturing Director', 'Healthcare Representative', 'Manager', 'Sales Representative', 'Research Director', or 'Human Resources'
* `JobSatisfaction`: the employee's ranking of their satisfaction with their job. Minimum is 1, maximum is 4.
* `MaritalStatus`: 'Single', 'Married', or 'Divorced'
* `MonthlyIncome`: in dollars
* `NumCompaniesWorked`: the total number of companies the employee has worked at in their lifetime
* `OverTime`: whether or not the employee is eligible for overtime. 1 (yes) or 0 (no)
* `PercentSalaryHike`: for the employee's most recent salary increase, the percentage by which their salary increased
* `WorkLifeBalance`: employee's rating of their work-life balance. 1 (bad), 2 (good), 3 (better), 4 (best)
* `YearsAtCompany`: number of years employee has worked for IBM
* `YearsInCurrentRole`: number of years employee has worked for IBM in their current position

In [3]:
summary(hr)

      Age        Attrition            BusinessTravel   DailyRate     
 Min.   :18.00   No :1233   Non-Travel       : 150   Min.   : 102.0  
 1st Qu.:30.00   Yes: 237   Travel_Frequently: 277   1st Qu.: 465.0  
 Median :36.00              Travel_Rarely    :1043   Median : 802.0  
 Mean   :36.92                                       Mean   : 802.5  
 3rd Qu.:43.00                                       3rd Qu.:1157.0  
 Max.   :60.00                                       Max.   :1499.0  
                                                                     
                  Department  DistanceFromHome   Education    
 Human Resources       : 63   Min.   : 1.000   Min.   :1.000  
 Research & Development:961   1st Qu.: 2.000   1st Qu.:2.000  
 Sales                 :446   Median : 7.000   Median :3.000  
                              Mean   : 9.193   Mean   :2.913  
                              3rd Qu.:14.000   3rd Qu.:4.000  
                              Max.   :29.000   Max.   :5.000  

In [41]:
# replace the ... with the name of the column you want to visualize
# capitalization counts!
my_column = "..."

# run this cell to see the distribution of values for your column
hist(hr[, my_column])

boxplot(hr[,my_column])

ERROR: Error in `[.data.frame`(hr, , my_column): undefined columns selected


## <a class="sectionlogistic"> Logistic Regression </a>

In [45]:
hr

In [39]:
numeric_data = subset(x=hr, subset=)

In [44]:
install.packages("Amelia")
library(Amelia)
library(mlbench)
missmap(hr, col=c("blue", "red"), legend=FALSE)

Warning message:
“dependency ‘foreign’ is not available”Warning message in install.packages("Amelia"):
“installation of package ‘Amelia’ had non-zero exit status”Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


ERROR: Error in library(Amelia): there is no package called ‘Amelia’


In [ ]:

library(corrplot)
correlations <- cor(Smarket[,1:8])
corrplot(correlations, method="circle")